In [1]:
#Import Libraries
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sgt import SGT
import os 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import math
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


2023-02-19 17:05:27.456609: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-19 17:05:27.958841: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-19 17:05:27.958888: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-19 17:05:27.958892: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
#Read Corpus
path = 'TOP_CON/Healthy/'
folder_names = os.listdir(path)

def load_files(folder_names, path):
    l = []
    for folder in folder_names: 
        p = path + folder + '/'
        p_os = os.listdir(p)
        for filename in p_os:
            if filename!='.ipynb_checkpoints': 
                new_path = p + filename 
                x = pd.read_csv(new_path)
                l.append(x)
    return l
corpus_healthy = load_files(folder_names, path)

path = 'TOP_CON/Glaucoma/'
folder_names = os.listdir(path)

corpus_diseased = load_files(folder_names, path)



In [3]:
def cat_encode(x,y):
    arr = []
    for i,j in zip(x,y):
        if 0<i and 0.6 > i and 0<j and 0.6 > j: 
            arr.append('A')
        elif 0<i and 0.25 > i and 0.6<j and 1 > j: 
            arr.append('B')
        elif 0.25<i and 0.48 > i and 0.6<j and 1 > j: 
            arr.append('C')
        elif 0.48<i and 0.6 > i and 0.6<j and 1 > j: 
            arr.append('D')
        elif 0.6<i and 1 > i and 0<j and 0.6 > j: 
            arr.append('E')
        elif 0.6<i and 0.77 > i and 0.6<j and 1 > j: 
            arr.append('F')
        elif 0.77<i and 1 > i and 0.6<j and 1 > j: 
            arr.append('G')
        else: 
            arr.append('error')

    return arr
    
def make_corpus(corpus_new):
    A = np.array(corpus_new.fixation_id[corpus_new.letters=='A'])
    B = np.array(corpus_new.fixation_id[corpus_new.letters=='B'])
    C = np.array(corpus_new.fixation_id[corpus_new.letters=='C'])
    D = np.array(corpus_new.fixation_id[corpus_new.letters=='D'])
    E = np.array(corpus_new.fixation_id[corpus_new.letters=='E'])
    F = np.array(corpus_new.fixation_id[corpus_new.letters=='F'])
    G = np.array(corpus_new.fixation_id[corpus_new.letters=='G'])
    corpus = pd.DataFrame([[1, A], 
                           [2, B],
                           [3, C], 
                           [4, D], 
                           [5, E], 
                           [6, F],
                           [7, G]], columns=['id', 'sequence'])
    return corpus

l = []
for i in corpus_healthy: 
    arr = cat_encode(i.norm_pos_x, i.norm_pos_y)
    
    i['letters'] = arr
    i_new = i[['letters', 'fixation_id']]
    i_new = make_corpus(i_new)
    l.append(i_new)
    
for i in corpus_diseased: 
    arr = cat_encode(i.norm_pos_x, i.norm_pos_y)
    i['letters'] = arr
    i_new = i[['letters', 'fixation_id']]
    i_new = make_corpus(i_new)
    l.append(i_new)
    


In [4]:
e = []
for i in l: 
    sgt_ = SGT(kappa=1, 
               lengthsensitive=False, 
               mode='multiprocessing')
    sgtembedding_df = sgt_.fit_transform(i)
    sgtembedding_df = sgtembedding_df.set_index('id')
    e.append(sgtembedding_df)

print(len(e))

INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 31 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

In [5]:
df1 = pd.DataFrame()
for i in e: 
    pca = PCA(n_components=2)
    pca.fit(i)
    X=pca.transform(i)
    df = pd.DataFrame(data=X, columns=['x1', 'x2'])
    df_new = pd.DataFrame([[df.iloc[0,0], df.iloc[0,1], 
                   df.iloc[1,0], df.iloc[1,1], 
                   df.iloc[2,0], df.iloc[2,1], 
                  df.iloc[3,0], df.iloc[3,1], 
                  df.iloc[4,0], df.iloc[4,1], 
                   df.iloc[5,0], df.iloc[5,1], 
                  df.iloc[6,0], df.iloc[6,1]]], 
                 columns = ['0x1', '0x2', '1x1', '1x2', '2x1', '2x2', '3x1', '3x2', '4x1', '4x2', '5x1', '5x3', '6x1', '6x2'])
    df1 = pd.concat([df1, df_new])
df1 = df1.reset_index().drop(["index"], axis = 1)
labels = [1]*80 + [0]*80

df1['labels'] = labels
df1

,0x1,0x2,1x1,1x2,2x1,2x2,3x1,3x2,4x1,4x2,5x1,5x3,6x1,6x2,labels
0,0.958814,0.002904,-0.157562,-0.048826,-0.157562,-0.048826,-0.157562,-0.048826,-0.157562,-0.048826,-0.157562,-0.048826,-0.171003,0.241227,1
1,0.492505,0.013559,-0.076684,-0.050352,-0.076684,-0.050352,-0.076684,-0.050352,-0.076684,-0.050352,-0.109084,0.238200,-0.076684,-0.050352,1
2,1.040909,0.045676,-0.152488,-0.110288,-0.152488,-0.110288,-0.177155,-0.188429,-0.253801,0.583907,-0.152488,-0.110288,-0.152488,-0.110288,1
3,0.583557,0.007808,-0.083858,-0.024879,-0.083858,-0.024879,-0.083858,-0.024879,-0.119260,0.287432,-0.106362,-0.110301,-0.106362,-0.110301,1
4,0.398856,0.211806,-0.013757,-0.075144,-0.013757,-0.075144,-0.013757,-0.075144,-0.026890,-0.208519,-0.316938,0.297288,-0.013757,-0.075144,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,1.054057,0.001520,-0.169547,-0.016370,-0.169547,-0.016370,-0.189305,0.281765,-0.186566,-0.217804,-0.169547,-0.016370,-0.169547,-0.016370,0
156,0.687810,-0.021888,-0.116309,0.004327,-0.147840,-0.238143,-0.116309,0.004327,-0.074732,0.242721,-0.116309,0.004327,-0.116309,0.004327,0
157,0.313248,0.055010,-0.037692,-0.055334,-0.037692,-0.055334,-0.037692,-0.055334,-0.124787,0.221661,-0.037692,-0.055334,-0.037692,-0.055334,0
158,1.261763,0.311580,-0.157908,-0.316158,-0.182191,-0.378833,-0.121104,-0.229561,-0.549556,1.091942,-0.129901,-0.249410,-0.121104,-0.229561,0


In [6]:
# df1 = df1.sample(frac=1)
labels = df1.labels
df_new = df1.drop(['labels'], axis =1 )
# display(df_new)


# train = df_new[0:120]
# train_labels = np.array(labels[0:120])

# test = df_new[120:]
# test_labels = np.array(labels[120:])

X_train, X_test, y_train, y_test = train_test_split(df_new, labels, test_size=0.2, random_state=42)



In [19]:
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []
    
    nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

In [20]:
def createmodel(n_layers, first_layer_nodes, activation_func, loss_func, last_layer_nodes=4):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(first_layer_nodes, input_dim=X_train.shape[1], activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss=loss_func, metrics = ["accuracy"]) #note: metrics could also be 'mse'
    
    return model

##Wrap model into scikit-learn
model =  KerasClassifier(build_fn=createmodel, verbose = False)  


/tmp/ipykernel_535337/1884772866.py:17: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model =  KerasClassifier(build_fn=createmodel, verbose = False)


In [21]:
activation_funcs = ['relu', 'tanh'] 
loss_funcs = ['binary_crossentropy']
param_grid = dict(n_layers=[3], first_layer_nodes = [64,32,16],activation_func = activation_funcs, loss_func = loss_funcs, batch_size = [10,50,100], epochs = [10,20,30])
grid = GridSearchCV(estimator = model, param_grid = param_grid)


In [22]:
grid.fit(X_train,y_train)

GridSearchCV(estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7fe676f236a0>,
             param_grid={'activation_func': ['relu', 'tanh'],
                         'batch_size': [10, 50, 100], 'epochs': [10, 20, 30],
                         'first_layer_nodes': [64, 32, 16],
                         'loss_func': ['binary_crossentropy'],
                         'n_layers': [3]})

In [23]:
print(grid.best_score_)
print(grid.best_params_)

0.564000004529953
{'activation_func': 'relu', 'batch_size': 50, 'epochs': 10, 'first_layer_nodes': 32, 'loss_func': 'binary_crossentropy', 'n_layers': 3}


In [26]:
model = Sequential()
model.add(Dense(32, input_shape=(14,), activation='relu'))
model.add(Dense(32, input_shape=(14,), activation='relu'))
model.add(Dense(32, input_shape=(14,), activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [27]:
# Compile neural network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [28]:

model.fit(
  X_train, # training data
  y_train, # training targets
  epochs=10,
  batch_size=50,
)

Epoch 1/10
3/3 [==============================] - 0s 2ms/step - loss: 0.6914 - accuracy: 0.5000
Epoch 2/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6871 - accuracy: 0.5234
Epoch 3/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6842 - accuracy: 0.5234
Epoch 4/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6820 - accuracy: 0.5312
Epoch 5/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6797 - accuracy: 0.5469
Epoch 6/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6774 - accuracy: 0.5469
Epoch 7/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6755 - accuracy: 0.5781
Epoch 8/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6732 - accuracy: 0.6094
Epoch 9/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6710 - accuracy: 0.6484
Epoch 10/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6691 - accuracy: 0.6406


In [29]:
y_pred = model.predict(X_test)
print(y_pred.shape)
rounded = [int(round(x[0])) for x in y_pred]
print(rounded)
from sklearn.metrics import accuracy_score
a = accuracy_score(rounded,y_test)
print(y_test)
print('Accuracy is:', a*100)

1/1 [==============================] - 0s 32ms/step
(32, 1)
[0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0]
105    0
108    0
141    0
55     1
94     0
29     1
101    0
51     1
100    0
142    0
19     1
84     0
15     1
66     1
24     1
30     1
128    0
147    0
98     0
16     1
75     1
18     1
12     1
9      1
31     1
151    0
97     0
56     1
131    0
104    0
136    0
78     1
Name: labels, dtype: int64
Accuracy is: 53.125
